In [1]:
%cd ..

/Users/floydluo/Library/CloudStorage/GoogleDrive-jjluo@terpmail.umd.edu/.shortcut-targets-by-id/1qNzMmGHCg5Xa63Vw3aKbZdMXvkfT2CgC/MedStar/MS_CODE/FieldNN


# Data Point

In [2]:
from recfldgrn.datapoint import PatientDP, RANGE_SIZE

PID = 'P4'
rec_folder = 'data/ProcData/RecFolder/'
Pat = PatientDP(PID, rec_folder, RANGE_SIZE)
Pat

In [3]:
####################
RecName = 'Diag'
####################

df = Pat.get_df_rec(RecName)
df

,PID,ECID,RID,DT,Value
0,P4,EC125,238249,2023-03-18,N97.5
1,P4,EC125,158555,2023-03-22,X4.5
2,P4,EC125,158547,2023-04-02,P36.5
3,P4,EC24,238252,2023-03-18,C91.0
4,P4,EC24,145433,2023-03-22,H48.5
...,...,...,...,...,...
79,P4,EC121,158548,2023-04-16,R28.5
80,P4,EC196,248719,NaT,NaN
81,P4,EC203,238255,2023-04-04,P40.5
82,P4,EC204,145439,2023-04-12,U55.0


# Task Selection

In [4]:
TaskName = 'Task2YearXXX'

ELIG_PAT_SET = [f'P{i}' for i in range(0, 8)]
ELIG_PAT_SET

['P0', 'P1', 'P2', 'P3', 'P4', 'P5', 'P6', 'P7']

In [5]:
import os
import pandas as pd

Tensor_folder = 'data/ProcData/TensorFolder'
TaskTensor_folder = os.path.join(Tensor_folder, TaskName)

if not os.path.exists(TaskTensor_folder): os.makedirs(TaskTensor_folder)

print(TaskTensor_folder)

data/ProcData/TensorFolder/Task2YearXXX


In [6]:
######################### Change this
RecName = 'Diag'
######################### 


df = pd.DataFrame()
for PID in ELIG_PAT_SET:
    Pat = PatientDP(PID, rec_folder, RANGE_SIZE)
    dfx = Pat.get_df_rec(RecName)
    
    ###################################
    # TODO: make some operation to select good dfx
    dfx = dfx 
    ###################################
    
    df = pd.concat([df, dfx])
    
print(df.shape)
df

(934, 5)


,PID,ECID,RID,DT,Value
0,P0,EC18,200972,2023-04-10,D14.0
1,P0,EC18,227676,2023-04-13,J19.5
2,P0,EC18,214361,2023-04-16,J74.5
3,P0,EC18,200975,2023-04-08,I94.0
4,P0,EC140,161119,2023-03-19,N91.5
...,...,...,...,...,...
20,P6,EC173,174622,2023-04-05,R80.5
0,P7,EC17,209056,2023-03-18,S37.5
1,P7,EC17,209055,2023-03-18,J24.5
2,P7,EC17,248716,2023-03-24,J74.5


# Load Meta Info

In [7]:
import os
import pandas as pd

fldgrn_folder = 'data/ProcData/FldGrnInfo'

fullfldgrn_file = os.path.join(fldgrn_folder, RecName + '.p')

df_FieldGrainInfo = pd.read_pickle(fullfldgrn_file)
df_FieldGrainInfo

,RecName,prefix_ids,focal_ids,field_list,field,grain,field2grain,recfield2grain,Vocab
0,Diag,"[PID, ECID]",[RID],"[Value, DT]",Value,DiagDftGrn,Value-DiagDftGrn,Diag@Value-DiagDftGrn,"{'idx2v': {0: '_padding', 1: '_missing', 2: 'C..."
1,Diag,"[PID, ECID]",[RID],"[Value, DT]",DT,DTDftGrn,DT-DTDftGrn,Diag@DT-DTDftGrn,"{'idx2v': {0: '_padding', 1: '_missing', 2: 'Y..."


# RecFldGrn `Diag@Value-DiagDftGrn`

In [8]:
from recfldgrn.grains import Field2Grain_2_get_GrnStrList_Fn
from recfldgrn.grains import Field2Grain_2_get_GrnIdxList_Fn
from recfldgrn.grains import Field2Grain_2_get_GrnWgtList_Fn
from recfldgrn.utils import generate_grain_vocab_info
from recfldgrn.utils import get_compressed_df
from recfldgrn.datapoint import write_df_to_folders

In [9]:
from recfldgrn.utils import get_highorder_input_idx

In [10]:
TaskTensor_folder

'data/ProcData/TensorFolder/Task2YearXXX'

In [11]:
######################### Change this
recfield2grain = 'Diag@Value-DiagDftGrn'
#########################


d = df_FieldGrainInfo[df_FieldGrainInfo['recfield2grain'] == recfield2grain].iloc[0]
v2idx = d['Vocab']['v2idx']
prefix_ids = d['prefix_ids']
focal_ids = d['focal_ids']

df_p = get_highorder_input_idx(df, recfield2grain, 
                               v2idx, prefix_ids, focal_ids,
                               Field2Grain_2_get_GrnIdxList_Fn, 
                               Field2Grain_2_get_GrnWgtList_Fn)



fullTaskTensor_folder = os.path.join(TaskTensor_folder, recfield2grain)
print(df.shape, fullTaskTensor_folder)
if not os.path.exists(fullTaskTensor_folder): os.makedirs(fullTaskTensor_folder)
write_df_to_folders(recfield2grain, fullTaskTensor_folder, df_p)

df_p

(934, 6) data/ProcData/TensorFolder/Task2YearXXX/Diag@Value-DiagDftGrn


,PID,P-EC-Diag@Value-DiagDftGrn_idx
0,P0,"[[[17, 630, 3], [12, 631, 2]], [[23, 625, 3], ..."
1,P1,"[[[7, 44, 2], [8, 133, 2], [18, 122, 3], [14, ..."
2,P2,"[[[23, 765, 3], [6, 766, 3], [17, 767, 2], [19..."
3,P3,"[[[18, 706, 2], [16, 127, 3], [24, 707, 3], [1..."
4,P4,"[[[7, 191, 2], [5, 78, 3]], [[21, 364, 2], [19..."
5,P5,"[[[13, 361, 3], [21, 359, 2], [24, 376, 3], [2..."
6,P6,"[[[5, 294, 2], [26, 281, 3], [23, 301, 2]], [[..."
7,P7,"[[[17, 800, 2], [9, 47, 2], [9, 37, 2], [14, 3..."


# RecFldGrn `Diag@DT-DTDftGrn`

In [12]:
######################### Change this
recfield2grain = 'Diag@DT-DTDftGrn'
#########################

d = df_FieldGrainInfo[df_FieldGrainInfo['recfield2grain'] == recfield2grain].iloc[0]
v2idx = d['Vocab']['v2idx']
prefix_ids = d['prefix_ids']
focal_ids = d['focal_ids']

df_p = get_highorder_input_idx(df, recfield2grain, 
                               v2idx, prefix_ids, focal_ids,
                               Field2Grain_2_get_GrnIdxList_Fn, 
                               Field2Grain_2_get_GrnWgtList_Fn)


fullTaskTensor_folder = os.path.join(TaskTensor_folder, recfield2grain)
print(df.shape, fullTaskTensor_folder)
if not os.path.exists(fullTaskTensor_folder): os.makedirs(fullTaskTensor_folder)
write_df_to_folders(recfield2grain, fullTaskTensor_folder, df_p)

df_p

(934, 7) data/ProcData/TensorFolder/Task2YearXXX/Diag@DT-DTDftGrn


,PID,P-EC-Diag@DT-DTDftGrn_idx
0,P0,"[[[16, 26, 61, 69, 79, 94, 138], [16, 27, 45, ..."
1,P1,"[[[16, 26, 54, 69, 79, 93, 138], [16, 26, 58, ..."
2,P2,"[[[16, 26, 56, 69, 74, 94, 138], [16, 26, 67, ..."
3,P3,"[[[16, 26, 54, 69, 79, 93, 138], [16, 26, 64, ..."
4,P4,"[[[16, 26, 61, 69, 79, 94, 138], [16, 26, 66, ..."
5,P5,"[[[16, 26, 57, 69, 75, 94, 138], [16, 26, 58, ..."
6,P6,"[[[16, 26, 55, 69, 80, 93, 138], [16, 27, 48, ..."
7,P7,"[[[16, 26, 54, 69, 79, 93, 138], [16, 26, 54, ..."
